<a href="https://colab.research.google.com/github/Technocolabs100/Spotify-Million-Songs-Playlist-Collaborative-Filtering-Method/blob/Samrat-doi/Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.cluster import KMeans
from scipy.spatial.distance import euclidean
from collections import defaultdict
from sklearn.model_selection import cross_val_score
from sklearn.utils import resample
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/Colab Notebooks")

Mounted at /content/drive


In [ ]:
df= pd.read_csv('data_extracted.csv')
df_spotify = df.copy()

In [4]:
df_spotify =df_spotify.drop(['description','collaborative'], axis =1)
df =df.drop(['description','collaborative'], axis =1)

In [5]:
df_spotify = df_spotify.dropna()
df = df.dropna()

In [6]:
df = df.drop_duplicates(['track_uri'])

df_spotify = df_spotify.drop_duplicates(['track_uri'])
df_spotify_tracks = df_spotify['track_uri']
df_spotify_track_names = df_spotify['track_name']

# Cleaning the data frame and removing features we don't need
df_spotify = df_spotify.drop_duplicates(['track_uri']).reset_index()
df_spotify_tracks = df_spotify['track_uri']
df_spotify_track_names = df_spotify['track_name']

# We will be using the clean df to implement our model
df_spotify_clean = df_spotify.drop(columns=['name', 'pid', 'modified_at', 'num_tracks',
       'num_albums', 'num_followers', 'num_edits', 'playlist_duration_ms',
       'num_artists', 'pos', 'artist_name', 'track_uri', 'artist_uri',
       'track_name', 'album_uri','index' ,'album_name', 'Name',
       'Album', 'Artist', 'Release_date'])

In [7]:
scaler = MinMaxScaler()
scaler.fit(df_spotify_clean)
df_spotify_clean_scaled = scaler.transform(df_spotify_clean)
df_spotify_clean_scaled = pd.DataFrame(df_spotify_clean_scaled)

In [8]:
df_spotify_scaledd = df_spotify_clean_scaled.iloc[:30000,:]

In [9]:
from sklearn.metrics.pairwise import cosine_similarity 
df_spotify_cosine = cosine_similarity(df_spotify_scaledd)

In [10]:
df_spotify_cosine = pd.DataFrame(df_spotify_cosine)
df_spotify_clean_scaled['track_uri'] = df_spotify['track_uri']
df_spotify_clean_scaled.head()

,0,1,2,3,4,5,6,7,8,9,10,11,track_uri
0,0.042968,0.042968,0.728261,0.031225,0.912210,0.813,0.006977,0.0471,0.842733,0.125780,0.540594,0.8,0UaMYEvWZi0ZqiDOoHU3YI
1,0.037653,0.037653,0.880435,0.025000,0.781029,0.838,0.025025,0.2420,0.893573,0.118503,0.616339,0.8,6I9VzXrHxO9rA9A5euc8Ak
2,0.044686,0.044686,0.228261,0.002390,0.670030,0.758,0.000000,0.0598,0.851050,0.218295,0.427693,0.8,0WqIKmW4BTrj3eJFmnCKMv
3,0.050621,0.050621,0.826087,0.201807,0.900101,0.714,0.000234,0.0521,0.859462,0.146570,0.435074,0.8,1AWQoqb9bSvzTjaLralEkT
4,0.043108,0.043108,0.000000,0.056325,0.860747,0.606,0.000000,0.3130,0.882707,0.074116,0.408303,0.8,1lzr43nnXAijIGYnCT8M8H


In [ ]:
playlist = df_spotify_clean_scaled.iloc[:10,:]

In [12]:
def recommend_songs(uri, df_cosine_similarity, df, num_tracks = 10):
    
    index = df.index[df['track_uri'] == uri][0]
    similarities = df_cosine_similarity.iloc[:, index].sort_values(ascending=False)
    final_indices = list(similarities[1:num_tracks].index)
    songs_recommended = df[['track_name','artist_name','track_uri']].iloc[final_indices]
    return songs_recommended

In [22]:
print('Song: ')
index = df.index[df['track_uri']== '0UaMYEvWZi0ZqiDOoHU3YI'][0]
song = df[['track_name','artist_name','track_uri']].iloc[index]
print(song)
print('Recommended Songs: ')
recommend_songs('0UaMYEvWZi0ZqiDOoHU3YI', df_spotify_cosine, df,num_tracks = 9)

Song: 
track_name     Lose Control (feat. Ciara & Fat Man Scoop)
artist_name                                 Missy Elliott
track_uri                          0UaMYEvWZi0ZqiDOoHU3YI
Name: 0, dtype: object
Recommended Songs: 


,track_name,artist_name,track_uri
14794,Bust A Move,Young MC,3IeCYkdKI55mNMl2o4vX0k
8855,"I Don't Like It, I Love It (feat. Robin Thicke...",Flo Rida,2S5LNtRVRPbXk01yRQ14sZ
2905,It's Tricky,Run–D.M.C.,6jBCehpNMkwFVF3dz4nLIW
28792,Just Fine,Mary J. Blige,33vzOPcd9FRirYGlCu32x4
52173,Like I Love You,Justin Timberlake,6W2Ef5Ph6ILTUAedoQ3QIv
14234,Brick House,Commodores,5VJjhHyG8NZ5xdgG6uTb3P
5103,Word Up,Cameo,7LoGfKBAaOl0nxhodJ1240
7826,Calabria 2008,Natasja,4MlfMyLq5REr81WoHM6y0N


In [14]:
def generate_playlist(playlist, df_spotify_cosine, df, num_playlist = 10):
  indices = list(df[df['track_uri'].isin(playlist['track_uri'])].index)
  #print("Playlist provided:")
  #display(info_df.iloc[indices][['track_name','artist_name','track_uri']])

  x = df_spotify_cosine.iloc[:,indices]

  my_list = []
  for i in range(len(indices)):
    my_list.append(list(x.iloc[:, i].sort_values(ascending = False)[1:num_playlist + 1].index))
  #my_list = np.array(my_list).transpose()
    
  rec_playlist = []
  for i in my_list:
    rec_playlist.append(df[['track_name','artist_name','track_uri']].iloc[i])
    
  return my_list

In [ ]:
recommendations = generate_playlist(playlist, df_spotify_cosine, df)

In [16]:
x = recommendations
indices = [element for sub in x for element in sub]
rec_playlist = []
for i in indices:
  rec_playlist.append(df[['track_name','artist_name','track_uri']].iloc[i])

recommendations = pd.DataFrame(rec_playlist)
recommendations

,track_name,artist_name,track_uri
14794,Bust A Move,Young MC,3IeCYkdKI55mNMl2o4vX0k
8855,"I Don't Like It, I Love It (feat. Robin Thicke...",Flo Rida,2S5LNtRVRPbXk01yRQ14sZ
2905,It's Tricky,Run–D.M.C.,6jBCehpNMkwFVF3dz4nLIW
28792,Just Fine,Mary J. Blige,33vzOPcd9FRirYGlCu32x4
52173,Like I Love You,Justin Timberlake,6W2Ef5Ph6ILTUAedoQ3QIv
...,...,...,...
551,Remember The Name (feat. Styles Of Beyond),Fort Minor,6ndmKwWqMozN2tcZqzCX4K
6304,La Gozadera,Gente De Zona,0OMRAvrtLWE2TvcXorRiB9
3273,Dirt On My Boots,Jon Pardi,7f08qp3HROTkfrtNpiYGGo
3019,Swalla (feat. Nicki Minaj & Ty Dolla $ign),Jason Derulo,6kex4EBAj0WHXDKZMEJaaF
